In [4]:
import cv2
import time
import os
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
draw_utils = mp.solutions.drawing_utils

# Smart save path
if not os.path.exists("image1.jpg"):
    img_path = "image1.jpg"
elif not os.path.exists("image2.jpg"):
    img_path = "image2.jpg"
else:
    print("⚠️ Both images already exist.")
    exit()

cap = cv2.VideoCapture(0)
print("📸 Capturing will start in 10 seconds... Stand straight!")
time.sleep(10)

if not cap.isOpened():
    print("Cannot access camera.")
    exit()

print("📷 Camera started... Waiting for stable standing posture...")

stable_start_time = None
required_duration = 3  # seconds to hold posture

# Landmarks we require to be visible (upper body)
required_landmarks = [
    mp_pose.PoseLandmark.NOSE,
    mp_pose.PoseLandmark.LEFT_SHOULDER,
    mp_pose.PoseLandmark.RIGHT_SHOULDER,
    mp_pose.PoseLandmark.LEFT_ELBOW,
    mp_pose.PoseLandmark.RIGHT_ELBOW,
    mp_pose.PoseLandmark.LEFT_WRIST,
    mp_pose.PoseLandmark.RIGHT_WRIST,
    mp_pose.PoseLandmark.LEFT_HIP,
    mp_pose.PoseLandmark.RIGHT_HIP
]

def all_upper_body_visible(landmarks):
    """Check if all required upper body landmarks are visible (visibility > 0.7)."""
    return all(landmarks[l.value].visibility > 0.7 for l in required_landmarks)

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb)

    if result.pose_landmarks:
        draw_utils.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmarks = result.pose_landmarks.landmark

        if all_upper_body_visible(landmarks):
            ls = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            lh = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            la = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

            if abs(ls.x - lh.x) < 0.05 and abs(lh.x - la.x) < 0.05:
                if stable_start_time is None:
                    stable_start_time = time.time()
                elif time.time() - stable_start_time >= required_duration:
                    cv2.imwrite(img_path, frame)
                    print(f"✅ Stable standing posture detected — Image saved as '{img_path}'")
                    break
            else:
                stable_start_time = None
        else:
            stable_start_time = None  # Reset if not all upper body visible
    else:
        stable_start_time = None  # Reset if pose not detected

    cv2.imshow("Pose Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("❌ Cancelled by user.")
        break

cap.release()
cv2.destroyAllWindows()


📸 Capturing will start in 10 seconds... Stand straight!
📷 Camera started... Waiting for stable standing posture...
❌ Cancelled by user.


In [3]:
import sys
print(sys.executable)
!{sys.executable} -m pip install opencv-python mediapipe numpy==1.26.4


c:\Users\Admin\Documents\MitBots-Internship\venv310\Scripts\python.exe
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
  Using cached mediapipe-0.10.21-cp310-cp310-win_amd64.whl (51.0 MB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
  Using cached jaxlib-0.6.2-cp310-cp310-win_amd64.whl (57.9 MB)
  Using cached protobuf-4.25.8-cp310-abi3-win_amd64.whl (413 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
  Using cached jax-0.6.2-py3-none-any.whl (2.7 MB)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl (363 kB)
  Using cached matplotlib-3.10.5-cp310-cp310-win_amd64.whl (8.1 MB)
  Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl (45.3 MB)
  Using cached sentencepiece-0.2.0-cp310-cp310-win_amd64.whl (991 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
  Using cached

You should consider upgrading via the 'c:\Users\Admin\Documents\MitBots-Internship\venv310\Scripts\python.exe -m pip install --upgrade pip' command.
